In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features, reduce_mem_usage
from preprocessing import get_dummies
import datetime

from tqdm import tqdm
import time
import sys

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

### Data Load

In [53]:
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
df_train.set_index(key, inplace=True)
df_test.set_index(key, inplace=True)
df_trans = utils.read_df_pkl('../input/trans_last1*')
df_trans = reduce_mem_usage(df_trans)

df_trans = df_trans[[key, 'yyyymmdd', 'authorized_flag', 'category_1', 'month_lag']].drop_duplicates()

auth1 = df_trans[df_trans.authorized_flag==1]
auth0 = df_trans[df_trans.authorized_flag==0]

auth1_cat1 = auth1[auth1.category_1==1]
auth1_cat0 = auth1[auth1.category_1==0]
auth0_cat1 = auth0[auth0.category_1==1]
auth0_cat0 = auth0[auth0.category_1==0]
del auth1, auth0
gc.collect()

100%|██████████| 3/3 [00:03<00:00,  1.10s/it]


Mem. usage decreased to 553.72 Mb (34.8% reduction)


SystemExit: 

In [39]:
def date_diff_cal(df):
    df = df[[key, 'month_lag', 'yyyymmdd']].drop_duplicates()
    df.sort_values(by=[key, 'yyyymmdd'], inplace=True)
    df['yyyymmdd'] = pd.to_datetime( df['yyyymmdd'] )
    df['yyyymmdd_shift1'] = df[[key, 'yyyymmdd']].groupby(key)['yyyymmdd'].shift(1)
    df['date_diff'] = (df['yyyymmdd'] - df['yyyymmdd_shift1']).dt.days
    return df

auth1_cat0 = date_diff_cal(auth1_cat0)
auth1_cat1 = date_diff_cal(auth1_cat1)
auth0_cat0 = date_diff_cal(auth0_cat0)
auth0_cat1 = date_diff_cal(auth0_cat1)
auth1_cat0.head()

,card_id,month_lag,yyyymmdd,yyyymmdd_shift1,date_diff
145,C_ID_00007093c1,0,2018-02-02,NaT,NaN
143,C_ID_00007093c1,0,2018-02-06,2018-02-02,4.0
141,C_ID_00007093c1,0,2018-02-07,2018-02-06,1.0
151,C_ID_00007093c1,2,2018-04-03,2018-02-07,55.0
152,C_ID_00007093c1,2,2018-04-09,2018-04-03,6.0


### month_lag毎にdate_diffを集計

In [48]:
def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
df_train.set_index(key, inplace=True)
df_test.set_index(key, inplace=True)

last_type = 'last1'
fname_list = ['auth1_cat0' ,'auth1_cat1' ,'auth0_cat0' ,'auth0_cat1']
df_list = [auth1_cat0 ,auth1_cat1 ,auth0_cat0 ,auth0_cat1]

aggs = {}
aggs['date_diff'] = ['mean', 'std', 'min', 'max']

for fname, df in zip(fname_list, df_list):
    lag_list = df['month_lag'].drop_duplicates().values
    for month in lag_list:
        tmp = df.query(f"month_lag=={month}")
        tmp = tmp.groupby(key).agg(aggs)
        new_cols = get_new_columns(f'{last_type}_{fname}_month_lag{month}', aggs)
        tmp.columns = new_cols
        tmp.fillna(-1, inplace=True)
        df_train = df_train.join(tmp)
        df_test = df_test.join(tmp)
    
df_train.head()

100%|██████████| 3/3 [00:00<00:00, 130.99it/s]


0
2
1
0
1
2
0
0


,first_active_month,feature_1,feature_2,feature_3,target,last1_auth1_cat0_month_lag0_date_diff_mean,last1_auth1_cat0_month_lag0_date_diff_std,last1_auth1_cat0_month_lag0_date_diff_min,last1_auth1_cat0_month_lag0_date_diff_max,last1_auth1_cat0_month_lag2_date_diff_mean,last1_auth1_cat0_month_lag2_date_diff_std,last1_auth1_cat0_month_lag2_date_diff_min,last1_auth1_cat0_month_lag2_date_diff_max,last1_auth1_cat0_month_lag1_date_diff_mean,last1_auth1_cat0_month_lag1_date_diff_std,last1_auth1_cat0_month_lag1_date_diff_min,last1_auth1_cat0_month_lag1_date_diff_max,last1_auth1_cat1_month_lag0_date_diff_mean,last1_auth1_cat1_month_lag0_date_diff_std,last1_auth1_cat1_month_lag0_date_diff_min,last1_auth1_cat1_month_lag0_date_diff_max,last1_auth1_cat1_month_lag1_date_diff_mean,last1_auth1_cat1_month_lag1_date_diff_std,last1_auth1_cat1_month_lag1_date_diff_min,last1_auth1_cat1_month_lag1_date_diff_max,last1_auth1_cat1_month_lag2_date_diff_mean,last1_auth1_cat1_month_lag2_date_diff_std,last1_auth1_cat1_month_lag2_date_diff_min,last1_auth1_cat1_month_lag2_date_diff_max,last1_auth0_cat0_month_lag0_date_diff_mean,last1_auth0_cat0_month_lag0_date_diff_std,last1_auth0_cat0_month_lag0_date_diff_min,last1_auth0_cat0_month_lag0_date_diff_max,last1_auth0_cat1_month_lag0_date_diff_mean,last1_auth0_cat1_month_lag0_date_diff_std,last1_auth0_cat1_month_lag0_date_diff_min,last1_auth0_cat1_month_lag0_date_diff_max
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_92a2005557,2017-06,5,2,1,-0.820283,1.769231,1.012739,1.0,4.0,3.222222,2.386304,1.0,8.0,2.833333,2.790677,1.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C_ID_3d0044924f,2017-01,4,1,0,0.392913,1.250000,0.531610,1.0,3.0,26.500000,23.334524,10.0,43.0,2.500000,2.121320,1.0,4.0,-1.0,-1.000000,-1.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C_ID_d639edf6cd,2016-08,2,2,0,0.688056,-1.000000,-1.000000,-1.0,-1.0,60.000000,-1.000000,60.0,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C_ID_186d6a6901,2017-09,4,3,0,0.142495,3.125000,1.457738,1.0,5.0,6.000000,3.559026,3.0,11.0,12.000000,7.071068,7.0,17.0,7.5,9.192388,1.0,14.0,NaN,NaN,NaN,NaN,56.0,-1.0,56.0,56.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C_ID_cdbd2c0db2,2017-11,1,3,0,-0.159749,1.421053,0.692483,1.0,3.0,2.000000,1.414214,1.0,5.0,2.818182,2.227922,1.0,8.0,22.0,-1.000000,22.0,22.0,17.0,-1.0,17.0,17.0,20.0,-1.0,20.0,20.0,NaN,NaN,NaN,NaN,-1.0,-1.0,-1.0,-1.0


In [49]:
fname = '163_fe2'
ignore_features = ['first_active_month', 'card_id', target]

for col in df_train.columns:
    if col in ignore_features: continue
    if col.count('feature_'):continue
    if not(col.count('date_diff')):continue
    feat_train = df_train[col].astype('float32').values
    feat_test = df_test[col].astype('float32').values
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_train_{col}@', obj=feat_train)
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_test_{col}@', obj=feat_test)

### date_diffのmean, stdの集計をより公平にするため、月初と月末のdateを追加する

In [71]:
sys.exit()
month_fl = df_trans[[key, 'yyyymm']].drop_duplicates()
month_fl.dropna(inplace=True)
month_fl['yyyymm_01'] =  month_fl['yyyymm'].map(lambda x: str(x)+'-01')
month_fl['yyyymm_28'] =  month_fl['yyyymm'].map(lambda x: str(x)+'-28')
df_fl = pd.concat([ month_fl.rename(columns={'yyyymm_01':'yyyymmdd'})[[key, 'yyyymmdd']], month_fl.rename(columns={'yyyymm_28':'yyyymmdd'})[[key, 'yyyymmdd']] ], axis=0)
df_fl['value'] = 1
df_fl = df_fl.pivot_table(index=key, columns='yyyymmdd', values='value').fillna(1).stack()
df_fl = df_fl.to_frame().reset_index()
df_fl.drop(0, axis=1, inplace=True)

,card_id,yyyymmdd
0,C_ID_00007093c1,2017-02-01
1,C_ID_00007093c1,2017-02-28
2,C_ID_00007093c1,2017-03-01
3,C_ID_00007093c1,2017-03-28
4,C_ID_00007093c1,2017-04-01
5,C_ID_00007093c1,2017-04-28
6,C_ID_00007093c1,2017-05-01
7,C_ID_00007093c1,2017-05-28
8,C_ID_00007093c1,2017-06-01
9,C_ID_00007093c1,2017-06-28


In [73]:
def date_diff_cal(df):
    df = df[[key, 'yyyymmdd']].drop_duplicates()
    df = pd.concat([df, df_fl], axis=0)
    df.sort_values(by=[key, 'yyyymmdd'], inplace=True)
    df['yyyymmdd'] = pd.to_datetime( df['yyyymmdd'] )
    df['yyyymmdd_shift1'] = df[[key, 'yyyymmdd']].groupby(key)['yyyymmdd'].shift(1)
    df['date_diff_regular'] = (df['yyyymmdd'] - df['yyyymmdd_shift1']).dt.days
    return df

auth1_cat0 = date_diff_cal(auth1_cat0)
auth1_cat1 = date_diff_cal(auth1_cat1)
auth0_cat0 = date_diff_cal(auth0_cat0)
auth0_cat1 = date_diff_cal(auth0_cat1)
auth1_cat0.head()

,card_id,yyyymmdd,yyyymmdd_shift1,date_diff,date_diff_regular
0,C_ID_00007093c1,2017-02-01,NaT,NaN,NaN
1,C_ID_00007093c1,2017-02-28,2017-02-01,27.0,27.0
2,C_ID_00007093c1,2017-03-01,2017-02-28,1.0,1.0
3,C_ID_00007093c1,2017-03-28,2017-03-01,27.0,27.0
4,C_ID_00007093c1,2017-04-01,2017-03-28,4.0,4.0


### month_lag毎にdate_diffを集計

In [77]:
def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
df_train.set_index(key, inplace=True)
df_test.set_index(key, inplace=True)

last_type = 'last1'
fname_list = ['auth1_cat0' ,'auth1_cat1' ,'auth0_cat0' ,'auth0_cat1']
df_list = [auth1_cat0 ,auth1_cat1 ,auth0_cat0 ,auth0_cat1]

aggs = {}
aggs['date_diff_regular'] = ['mean', 'std']

for fname, tmp in zip(fname_list, df_list):
    tmp = tmp.groupby(key).agg(aggs)
    new_cols = get_new_columns(f'{last_type}_{fname}', aggs)
    tmp.columns = new_cols
    tmp.fillna(-1, inplace=True)
    df_train = df_train.join(tmp)
    df_test = df_test.join(tmp)
    
df_train.head()

100%|██████████| 3/3 [00:00<00:00, 160.28it/s]


,first_active_month,feature_1,feature_2,feature_3,target,last1_auth1_cat0_date_diff_regular_mean,last1_auth1_cat0_date_diff_regular_std,last1_auth1_cat1_date_diff_regular_mean,last1_auth1_cat1_date_diff_regular_std,last1_auth0_cat0_date_diff_regular_mean,last1_auth0_cat0_date_diff_regular_std,last1_auth0_cat1_date_diff_regular_mean,last1_auth0_cat1_date_diff_regular_std
card_id,,,,,,,,,,,,,
C_ID_92a2005557,2017-06,5,2,1,-0.820283,7.062500,9.784787,15.551724,12.081823,15.551724,12.081823,15.551724,12.081823
C_ID_3d0044924f,2017-01,4,1,0,0.392913,7.775862,10.594674,15.033333,11.845218,15.551724,12.081823,15.551724,12.081823
C_ID_d639edf6cd,2016-08,2,2,0,0.688056,14.548387,12.230668,15.551724,12.081823,15.551724,12.081823,15.551724,12.081823
C_ID_186d6a6901,2017-09,4,3,0,0.142495,10.250000,10.696935,13.666667,11.373397,15.551724,12.081823,15.551724,12.081823
C_ID_cdbd2c0db2,2017-11,1,3,0,-0.159749,6.094595,9.372313,13.666667,11.381637,15.551724,12.081823,15.033333,12.132838


In [79]:
fname = '164_fe2'
ignore_features = ['first_active_month', 'card_id', target]

for col in df_train.columns:
    if col in ignore_features: continue
    if col.count('feature_'):continue
    if not(col.count('date_diff_regular')):continue
    feat_train = df_train[col].astype('float32').values
    feat_test = df_test[col].astype('float32').values
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_train_{col}@', obj=feat_train)
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_test_{col}@', obj=feat_test)